# Simplified EvilFlowers Metadata Postprocessing

This notebook demonstrates a simplified approach to read, clean, and enhance the metadata extracted by the EvilFlowers importer. It focuses on:

1. Loading data from progress.json
2. Sanitizing title, authors, and publisher
3. Creating a clean dataframe with single ISBN, DOI columns
4. Checking ISBNs against external databases
5. Fetching missing data
6. Creating indicators for found ISBNs

## Table of Contents
1. [Loading the Data](#loading-the-data)
2. [Data Cleanup](#data-cleanup)
   - [Sanitizing Titles](#sanitizing-titles)
   - [Sanitizing Author Names](#sanitizing-author-names)
   - [Sanitizing Publisher Names](#sanitizing-publisher-names)
   - [Cleaning ISBN/DOI Values](#cleaning-isbndoi-values)
3. [Creating a Clean DataFrame](#creating-a-clean-dataframe)
4. [Checking ISBNs Against External Databases](#checking-isbns-against-external-databases)
5. [Fetching Missing Data](#fetching-missing-data)
6. [Exporting Cleaned Data](#exporting-cleaned-data)


## Loading the Data

First, let's import the necessary libraries and load the metadata from the progress.json file.


In [10]:
import pandas as pd
import numpy as np
import re
import json
import ast
from pathlib import Path
import isbnlib
from tqdm.notebook import tqdm
import sys
import os

# Add the parent directory to the path so we can import from evilflowers_importer
sys.path.append(os.path.abspath('..'))

# Import the RegexExtractor from evilflowers_importer
from evilflowers_importer.ai_facade import RegexExtractor

# Load the data
progress_file = Path('../output/progress.json')
if progress_file.exists():
    df = pd.read_json(progress_file, orient="records")
    print(f"Loaded {len(df)} records from {progress_file}")
else:
    print(f"File not found: {progress_file}")
    df = pd.DataFrame()


Loaded 134 records from ../output/progress.json


## Data Cleanup

Let's clean up the data to make it more consistent and usable.


### Sanitizing Titles

First, let's clean up the titles to ensure consistent capitalization and formatting.


In [11]:
def clean_title(title):
    """Clean and standardize book titles."""
    if pd.isna(title) or title is None:
        return None

    # Convert to string if not already
    title = str(title).strip()

    # Remove extra whitespace
    title = re.sub(r'\s+', ' ', title)

    # Apply title case (capitalize first letter of each word)
    # But preserve common acronyms and special cases
    words = title.split()
    small_words = {'a', 'an', 'and', 'as', 'at', 'but', 'by', 'for', 'if', 'in', 'of', 'on', 'or', 'the', 'to', 'with'}

    for i, word in enumerate(words):
        # Always capitalize first and last word
        if i == 0 or i == len(words) - 1:
            words[i] = word.capitalize()
        # Check if it's an acronym (all uppercase)
        elif word.upper() == word and len(word) > 1:
            words[i] = word  # Keep acronyms as is
        # Check if it's a small word
        elif word.lower() in small_words:
            words[i] = word.lower()
        # Otherwise capitalize
        else:
            words[i] = word.capitalize()

    return ' '.join(words)

# Apply the function to standardize titles
df['title_cleaned'] = df['title'].apply(clean_title)

# Display the results
print("Original vs. Cleaned Titles:")
for i, (orig, cleaned) in enumerate(zip(df['title'].head(5), df['title_cleaned'].head(5))):
    print(f"{i+1}. Original: {orig}")
    print(f"   Cleaned: {cleaned}")


Original vs. Cleaned Titles:
1. Original: Projektovanie vozidiel
   Cleaned: Projektovanie Vozidiel
2. Original: Modelovanie a simulácie v dopravnej technike
   Cleaned: Modelovanie a Simulácie V Dopravnej Technike
3. Original: Tenzometria
   Cleaned: Tenzometria
4. Original: PRUŽNOSŤ A PEVNOSŤ - Riešené príklady
   Cleaned: Pružnosť a PEVNOSŤ - Riešené Príklady
5. Original: Termodynamické tabulky
   Cleaned: Termodynamické Tabulky


### Sanitizing Author Names

Next, let's ensure that the 'authors' field is consistently a list of strings.


In [12]:
def parse_authors(authors):
    """Convert authors to a list of strings, handling various input formats."""
    # Handle scalar NA values
    if isinstance(authors, (float, int, str, type(None))) and (pd.isna(authors) or authors is None):
        return []

    if isinstance(authors, list):
        return authors

    if isinstance(authors, str):
        # Try to parse as a Python list literal
        try:
            parsed = ast.literal_eval(authors)
            if isinstance(parsed, list):
                return parsed
        except (SyntaxError, ValueError):
            pass

        # If it's a string but not a list literal, split by common separators
        if '|' in authors:
            return [a.strip() for a in authors.split('|') if a.strip()]
        elif ';' in authors:
            return [a.strip() for a in authors.split(';') if a.strip()]
        elif ',' in authors:
            return [a.strip() for a in authors.split(',') if a.strip()]
        else:
            return [authors]

    return [str(authors)]

# Apply the function to standardize the authors field
df['authors_cleaned'] = df['authors'].apply(parse_authors)

# Display the results
print("Original vs. Cleaned Authors:")
for i, (orig, cleaned) in enumerate(zip(df['authors'].head(5), df['authors_cleaned'].head(5))):
    print(f"{i+1}. Original: {orig} (type: {type(orig).__name__})")
    print(f"   Cleaned: {cleaned} (type: {type(cleaned).__name__})")


Original vs. Cleaned Authors:
1. Original: ['Pavol Hudec'] (type: str)
   Cleaned: ['Pavol Hudec'] (type: list)
2. Original: ['Ľuboš Magdolen'] (type: str)
   Cleaned: ['Ľuboš Magdolen'] (type: list)
3. Original: ['Dr. h.c. mult. prof. Ing. František TREBUŇA, CSc.'
 'Ing. Peter SIVÁK, PhD.'] (type: str)
   Cleaned: ['Dr. h.c. mult. prof. Ing. František TREBUŇA, CSc.Ing. Peter SIVÁK, PhD.'] (type: list)
4. Original: ['Prof. Ing. Ján Syč-Milý, CSc.' 'kolektiv'] (type: str)
   Cleaned: ['Prof. Ing. Ján Syč-Milý, CSc.kolektiv'] (type: list)
5. Original: ['K. Ražnjevič'] (type: str)
   Cleaned: ['K. Ražnjevič'] (type: list)


### Sanitizing Publisher Names

Let's clean up the publisher names to ensure consistency.


In [13]:
def clean_publisher(publisher):
    """Clean and standardize publisher names."""
    if pd.isna(publisher) or publisher is None:
        return None

    # Convert to string if not already
    publisher = str(publisher).strip()

    # Remove extra whitespace
    publisher = re.sub(r'\s+', ' ', publisher)

    # Standardize common publisher name variations
    # This could be expanded based on your specific dataset
    publisher_map = {
        'SLOVENSK\\u00c1 TECHNICK\\u00c1 UNIVERZITA V BRATISLAVE': 'Slovenská Technická Universiteta v Bratislave',
        'SLOVENSK\u00c1 TECHNICK\u00c1 UNIVERZITA V BRATISLAVE': 'Slovenská Technická Universiteta v Bratislave',
        'Slovensk\u00e1 technick\u00e1 univerzita v Bratislave': 'Slovenská Technická Universiteta v Bratislave',
        'TECHNICK\\u00c1 UNIVERZITA V KO\\u0160ICIACH': 'Technická Univerzita v Košiciach',
        'TECHNICK\u00c1 UNIVERZITA V KO\u0160ICIACH': 'Technická Univerzita v Košiciach'
    }

    # Check if the publisher name is in our mapping
    if publisher in publisher_map:
        return publisher_map[publisher]

    return publisher

# Apply the function to standardize publisher names
df['publisher_cleaned'] = df['publisher'].apply(clean_publisher)

# Display the results
print("Original vs. Cleaned Publishers:")
for i, (orig, cleaned) in enumerate(zip(df['publisher'].head(5), df['publisher_cleaned'].head(5))):
    print(f"{i+1}. Original: {orig}")
    print(f"   Cleaned: {cleaned}")


Original vs. Cleaned Publishers:
1. Original: Slovenská technická univerzita v Bratislave
   Cleaned: Slovenská Technická Universiteta v Bratislave
2. Original: SLOVENSKÁ TECHNICKÁ UNIVERZITA V BRATISLAVE
   Cleaned: Slovenská Technická Universiteta v Bratislave
3. Original: TECHNICKÁ UNIVERZITA V KOŠICIACH
   Cleaned: Technická Univerzita v Košiciach
4. Original: VYDAVATEĽSTVO TECHNICKEJ A EKONOMICKEJ LITERATÚRY
   Cleaned: VYDAVATEĽSTVO TECHNICKEJ A EKONOMICKEJ LITERATÚRY
5. Original: VYDAVATEĽSTVO TECHNICKEJ A EKONOMICKEJ LITERATÚRY (BRATISLAVA)
   Cleaned: VYDAVATEĽSTVO TECHNICKEJ A EKONOMICKEJ LITERATÚRY (BRATISLAVA)


### Cleaning ISBN/DOI Values

Let's fix the ISBN and DOI fields to ensure they are in the correct format.


In [14]:
def clean_isbn(isbn):
    """Clean and standardize ISBN values."""
    if pd.isna(isbn) or isbn is None:
        return None

    # Convert to string
    isbn_str = str(isbn).strip()

    # Check if it's a year (common error in the dataset)
    if isbn_str.isdigit() and int(isbn_str) > 1900 and int(isbn_str) < 2100:
        return None

    # Remove non-alphanumeric characters except '-'
    isbn_str = re.sub(r'[^\w-]', '', isbn_str)

    # Check if it's a valid ISBN (simple check for length)
    if len(isbn_str) in [10, 13] and isbn_str.replace('-', '').isalnum():
        return isbn_str
    else:
        return None

def clean_doi(doi):
    """Clean and standardize DOI values."""
    if pd.isna(doi) or doi is None:
        return None

    # Convert to string
    doi_str = str(doi).strip()

    # Check if it matches the DOI pattern (10.xxxx/yyyy)
    if re.match(r'^10\.\d{4,9}/[-._;()/:A-Za-z0-9]+$', doi_str):
        return doi_str.lower()  # DOIs are case-insensitive
    else:
        return None

# Use RegexExtractor from evilflowers_importer for additional validation
def validate_isbn_with_regex(isbn):
    """Validate ISBN using RegexExtractor."""
    if pd.isna(isbn) or isbn is None:
        return None

    # Convert to string
    isbn_str = str(isbn)

    # Use RegexExtractor to validate
    validated = RegexExtractor.extract_isbn(isbn_str)
    return validated

def validate_doi_with_regex(doi):
    """Validate DOI using RegexExtractor."""
    if pd.isna(doi) or doi is None:
        return None

    # Convert to string
    doi_str = str(doi)

    # Use RegexExtractor to validate
    validated = RegexExtractor.extract_doi(doi_str)
    return validated

# Apply the functions to clean ISBN and DOI fields
# First clean the original fields
df['isbn_cleaned'] = df['isbn'].apply(clean_isbn)
df['doi_cleaned'] = df['doi'].apply(clean_doi)

# Then clean the LLM-extracted fields
df['llm_isbn_cleaned'] = df['llm_isbn'].apply(clean_isbn)
df['llm_doi_cleaned'] = df['llm_doi'].apply(clean_doi)

# Validate with RegexExtractor
df['isbn_validated'] = df['isbn'].apply(validate_isbn_with_regex)
df['llm_isbn_validated'] = df['llm_isbn'].apply(validate_isbn_with_regex)
df['doi_validated'] = df['doi'].apply(validate_doi_with_regex)
df['llm_doi_validated'] = df['llm_doi'].apply(validate_doi_with_regex)

# Create final ISBN and DOI fields, always prioritizing LLM values
df['final_isbn'] = df.apply(
    lambda row: row['llm_isbn_validated'] or row['llm_isbn_cleaned'] or row['llm_isbn'] or row['isbn_validated'] or row['isbn_cleaned'], 
    axis=1
)

df['final_doi'] = df.apply(
    lambda row: row['llm_doi_validated'] or row['llm_doi_cleaned'] or row['llm_doi'] or row['doi_validated'] or row['doi_cleaned'], 
    axis=1
)

# Display the results
print("Original vs. Final ISBN/DOI:")
for i in range(min(5, len(df))):
    print(f"{i+1}. ISBN: {df['isbn'].iloc[i]} -> {df['final_isbn'].iloc[i]}")
    print(f"   DOI: {df['doi'].iloc[i]} -> {df['final_doi'].iloc[i]}")


Original vs. Final ISBN/DOI:
1. ISBN: 2003 -> 802271853X
   DOI: None -> None
2. ISBN: 2011 -> 9788022736244
   DOI: None -> None
3. ISBN: 2012 -> 9788055313788
   DOI: None -> None
4. ISBN: 2343 -> 284J023936
   DOI: None -> None
5. ISBN: 1975 -> 1975
   DOI: None -> None


## Creating a Clean DataFrame

Now let's create a clean DataFrame with all our cleaned fields.


In [15]:
# Create a clean version of the DataFrame with all our cleaned fields
df_clean = pd.DataFrame()
df_clean['dirname'] = df['dirname']
df_clean['title'] = df['title_cleaned']
df_clean['authors'] = df['authors_cleaned']
df_clean['publisher'] = df['publisher_cleaned']
df_clean['isbn'] = df['final_isbn']
df_clean['doi'] = df['final_doi']
df_clean['summary'] = df['summary']
df_clean['cover_image'] = df['cover_image']

# Fix PDF path to point to the single PDF file inside stream_pdf folder
def fix_pdf_path(dirname, isbn):
    """
    Create the correct path to the PDF file in the stream_pdf folder.
    The PDF file is named OPACID_SJF_[ISBN].pdf and is located in the stream_pdf folder.
    """
    if pd.isna(dirname) or dirname is None:
        return None

    # Extract the base directory name (last part of the path)
    base_dirname = os.path.basename(dirname)

    # The PDF filename is derived from the directory name by replacing "CVI_" with ""
    # For example, if dirname is "/path/to/CVI_OPACID_SJF_8001029344",
    # the PDF file is named "OPACID_SJF_8001029344.pdf"
    if base_dirname.startswith("CVI_"):
        pdf_filename = base_dirname[4:] + ".pdf"
    else:
        pdf_filename = base_dirname + ".pdf"

    # Construct the path to the PDF file
    pdf_path = os.path.join(dirname, "stream_pdf", pdf_filename)

    return pdf_path

df_clean['pdf_path'] = df_clean.apply(lambda row: fix_pdf_path(row['dirname'], row['isbn']), axis=1)

# Fix the year field if it's numeric
def clean_year(year):
    if pd.isna(year) or year is None:
        return None

    year_str = str(year).strip()
    if year_str.isdigit() and int(year_str) > 1800 and int(year_str) < 2100:
        return year_str
    return None

df_clean['year'] = df['year'].apply(clean_year)

# Display the cleaned DataFrame
print("Cleaned DataFrame:")
df_clean.head()


Cleaned DataFrame:


,dirname,title,authors,publisher,isbn,doi,summary,cover_image,pdf_path,year
0,/Volumes/SJF/CVI_OPACID_SJF_802271853_X,Projektovanie Vozidiel,[Pavol Hudec],Slovenská Technická Universiteta v Bratislave,802271853X,None,"The text discusses a 2003 book titled ""MOTOROV...",/Volumes/SJF/CVI_OPACID_SJF_802271853_X/Cover/...,/Volumes/SJF/CVI_OPACID_SJF_802271853_X/stream...,2003
1,/Volumes/SJF/CVI_OPACID_SJF_9788022736244,Modelovanie a Simulácie V Dopravnej Technike,[Ľuboš Magdolen],Slovenská Technická Universiteta v Bratislave,9788022736244,None,This text discusses the importance and methods...,/Volumes/SJF/CVI_OPACID_SJF_9788022736244/Cove...,/Volumes/SJF/CVI_OPACID_SJF_9788022736244/stre...,2011
2,/Volumes/SJF/CVI_OPACID_SJF_9788055313788,Tenzometria,"[Dr. h.c. mult. prof. Ing. František TREBUŇA, ...",Technická Univerzita v Košiciach,9788055313788,None,The text discusses various strain measurement ...,/Volumes/SJF/CVI_OPACID_SJF_9788055313788/Cove...,/Volumes/SJF/CVI_OPACID_SJF_9788055313788/stre...,2012
3,/Volumes/SJF/CVI_OPACID_SJF_PRUZNOST_1988,Pružnosť a PEVNOSŤ - Riešené Príklady,"[Prof. Ing. Ján Syč-Milý, CSc.kolektiv]",VYDAVATEĽSTVO TECHNICKEJ A EKONOMICKEJ LITERATÚRY,284J023936,None,The text discusses various engineering problem...,/Volumes/SJF/CVI_OPACID_SJF_PRUZNOST_1988/Cove...,/Volumes/SJF/CVI_OPACID_SJF_PRUZNOST_1988/stre...,1988
4,/Volumes/SJF/CVI_OPACID_SJF_TERMODINAMICKE,Termodynamické Tabulky,[K. Ražnjevič],VYDAVATEĽSTVO TECHNICKEJ A EKONOMICKEJ LITERAT...,1975,None,The text contains two tables and a discussion ...,/Volumes/SJF/CVI_OPACID_SJF_TERMODINAMICKE/Cov...,/Volumes/SJF/CVI_OPACID_SJF_TERMODINAMICKE/str...,None


## Checking ISBNs Against External Databases

Now let's check the ISBNs against external databases and fetch missing data.


In [16]:
# Function to fetch metadata using ISBN
def fetch_isbn_metadata(isbn):
    """
    Fetch metadata for a book using its ISBN.
    Returns None if the ISBN is invalid or no metadata is found.
    """
    if not isbn or not isinstance(isbn, str):
        return None

    # Clean the ISBN (remove hyphens, spaces, etc.)
    try:
        clean_isbn = isbnlib.canonical(isbn)
        if not clean_isbn:
            return None
    except Exception:
        return None

    try:
        # Try to fetch metadata from multiple sources
        for service in ['goob', 'openl', 'wiki']:
            try:
                metadata = isbnlib.meta(clean_isbn, service=service)
                if metadata:
                    return metadata
            except Exception:
                continue
    except Exception:
        pass

    return None

# Test the function with a sample ISBN
sample_isbn = df_clean['isbn'].dropna().iloc[0] if not df_clean['isbn'].dropna().empty else None
if sample_isbn:
    print(f"Fetching metadata for sample ISBN: {sample_isbn}")
    sample_metadata = fetch_isbn_metadata(sample_isbn)
    print(f"Sample metadata: {sample_metadata}")
else:
    print("No valid ISBN found in the dataset for testing")


Fetching metadata for sample ISBN: 802271853X
Sample metadata: {'ISBN-13': '9788022718530', 'Title': 'Motorové vozidlá - Projektovanie vozidiel. III', 'Authors': ['Pavol Hudec'], 'Publisher': '', 'Year': '2003', 'Language': 'sk'}


## Fetching Missing Data

Now let's fetch metadata for all books with valid ISBNs and update our DataFrame.


In [17]:
# Add a column to indicate if ISBN was found in remote database
df_clean['isbn_found'] = False

# Apply the function to all rows with valid ISBNs and update the DataFrame
print("Fetching metadata for all books with valid ISBNs...")

# Get rows with valid ISBNs
valid_isbn_rows = df_clean[df_clean['isbn'].notna()]

# Create a progress bar
for i, (idx, row) in enumerate(tqdm(valid_isbn_rows.iterrows(), total=len(valid_isbn_rows), desc="Checking ISBNs")):
    isbn = row['isbn']
    metadata = fetch_isbn_metadata(isbn)

    if metadata:
        # Mark ISBN as found
        df_clean.at[idx, 'isbn_found'] = True

        # Update missing fields with fetched data
        if pd.isna(row['title']) or not row['title']:
            df_clean.at[idx, 'title'] = metadata.get('Title')

        if not row['authors'] or len(row['authors']) == 0:
            df_clean.at[idx, 'authors'] = metadata.get('Authors', [])

        if pd.isna(row['publisher']) or not row['publisher']:
            df_clean.at[idx, 'publisher'] = metadata.get('Publisher')

        if pd.isna(row['year']) or not row['year']:
            df_clean.at[idx, 'year'] = metadata.get('Year')

# Display statistics
found_count = df_clean['isbn_found'].sum()
total_isbns = df_clean['isbn'].notna().sum()
print(f"Found {found_count} out of {total_isbns} ISBNs in external databases ({found_count/total_isbns*100:.1f}%)")

# Display the updated DataFrame
print("\nUpdated DataFrame:")
df_clean.head()


Fetching metadata for all books with valid ISBNs...


Checking ISBNs:   0%|          | 0/134 [00:00<?, ?it/s]

[19:53:04] WARNING  DataNotFoundAtServiceError for                                                   ]8;id=36488;file:///Users/jdubec/Projects/EvilFlowers/evilflowers-webdav-import/.venv/lib/python3.13/site-packages/isbnlib/dev/webquery.py\webquery.py]8;;\:]8;id=740529;file:///Users/jdubec/Projects/EvilFlowers/evilflowers-webdav-import/.venv/lib/python3.13/site-packages/isbnlib/dev/webquery.py#40\40]8;;\
                    http://openlibrary.org/api/books?bibkeys=ISBN:9788022736244&format=json&jscmd=da               
                    ta                                                                                             

[19:53:05] CRITICAL ISBNLibHTTPError for                                                           ]8;id=711593;file:///Users/jdubec/Projects/EvilFlowers/evilflowers-webdav-import/.venv/lib/python3.13/site-packages/isbnlib/dev/webservice.py\webservice.py]8;;\:]8;id=807366;file:///Users/jdubec/Projects/EvilFlowers/evilflowers-webdav-import/.venv/lib/python3.13/site-packages/isbnlib/dev/webservice.py#62\62]8;;\
                    https://en.wikipedia.org/api/rest_v1/data/citation/mediawiki/9788022736244                     
                    with code 404 [Not Found]                                                                      

[19:53:08] WARNING  DataNotFoundAtServiceError for                                                   ]8;id=547245;file:///Users/jdubec/Projects/EvilFlowers/evilflowers-webdav-import/.venv/lib/python3.13/site-packages/isbnlib/dev/webquery.py\webquery.py]8;;\:]8;id=484917;file:///Users/jdubec/Projects/EvilFlowers/evilflowers-webdav-import/.venv/lib/python3.13/site-packages/isbnlib/dev/webquery.py#40\40]8;;\
                    http://openlibrary.org/api/books?bibkeys=ISBN:9788055313788&format=json&jscmd=da               
                    ta                                                                                             

...

[19:55:49] CRITICAL ISBNLibHTTPError for                                                           ]8;id=124397;file:///Users/jdubec/Projects/EvilFlowers/evilflowers-webdav-import/.venv/lib/python3.13/site-packages/isbnlib/dev/webservice.py\webservice.py]8;;\:]8;id=263861;file:///Users/jdubec/Projects/EvilFlowers/evilflowers-webdav-import/.venv/lib/python3.13/site-packages/isbnlib/dev/webservice.py#62\62]8;;\
                    https://en.wikipedia.org/api/rest_v1/data/citation/mediawiki/9788089313013                     
                    with code 404 [Not Found]                                                                      

[19:55:51] WARNING  DataNotFoundAtServiceError for                                                   ]8;id=778438;file:///Users/jdubec/Projects/EvilFlowers/evilflowers-webdav-import/.venv/lib/python3.13/site-packages/isbnlib/dev/webquery.py\webquery.py]8;;\:]8;id=611730;file:///Users/jdubec/Projects/EvilFlowers/evilflowers-webdav-import/.venv/lib/python3.13/site-packages/isbnlib/dev/webquery.py#40\40]8;;\
                    http://openlibrary.org/api/books?bibkeys=ISBN:9788089313358&format=json&jscmd=da               
                    ta                                                                                             

[19:55:52] CRITICAL ISBNLibHTTPError for                                                           ]8;id=477460;file:///Users/jdubec/Projects/EvilFlowers/evilflowers-webdav-import/.venv/lib/python3.13/site-packages/isbnlib/dev/webservice.py\webservice.py]8;;\:]8;id=960469;file:///Users/jdubec/Projects/EvilFlowers/evilflowers-webdav-import/.venv/lib/python3.13/site-packages/isbnlib/dev/webservice.py#62\62]8;;\
                    https://en.wikipedia.org/api/rest_v1/data/citation/mediawiki/9788089313358                     
                    with code 404 [Not Found]                                                                      

Found 28 out of 134 ISBNs in external databases (20.9%)

Updated DataFrame:


,dirname,title,authors,publisher,isbn,doi,summary,cover_image,pdf_path,year,isbn_found
0,/Volumes/SJF/CVI_OPACID_SJF_802271853_X,Projektovanie Vozidiel,[Pavol Hudec],Slovenská Technická Universiteta v Bratislave,802271853X,None,"The text discusses a 2003 book titled ""MOTOROV...",/Volumes/SJF/CVI_OPACID_SJF_802271853_X/Cover/...,/Volumes/SJF/CVI_OPACID_SJF_802271853_X/stream...,2003,True
1,/Volumes/SJF/CVI_OPACID_SJF_9788022736244,Modelovanie a Simulácie V Dopravnej Technike,[Ľuboš Magdolen],Slovenská Technická Universiteta v Bratislave,9788022736244,None,This text discusses the importance and methods...,/Volumes/SJF/CVI_OPACID_SJF_9788022736244/Cove...,/Volumes/SJF/CVI_OPACID_SJF_9788022736244/stre...,2011,False
2,/Volumes/SJF/CVI_OPACID_SJF_9788055313788,Tenzometria,"[Dr. h.c. mult. prof. Ing. František TREBUŇA, ...",Technická Univerzita v Košiciach,9788055313788,None,The text discusses various strain measurement ...,/Volumes/SJF/CVI_OPACID_SJF_9788055313788/Cove...,/Volumes/SJF/CVI_OPACID_SJF_9788055313788/stre...,2012,False
3,/Volumes/SJF/CVI_OPACID_SJF_PRUZNOST_1988,Pružnosť a PEVNOSŤ - Riešené Príklady,"[Prof. Ing. Ján Syč-Milý, CSc.kolektiv]",VYDAVATEĽSTVO TECHNICKEJ A EKONOMICKEJ LITERATÚRY,284J023936,None,The text discusses various engineering problem...,/Volumes/SJF/CVI_OPACID_SJF_PRUZNOST_1988/Cove...,/Volumes/SJF/CVI_OPACID_SJF_PRUZNOST_1988/stre...,1988,False
4,/Volumes/SJF/CVI_OPACID_SJF_TERMODINAMICKE,Termodynamické Tabulky,[K. Ražnjevič],VYDAVATEĽSTVO TECHNICKEJ A EKONOMICKEJ LITERAT...,1975,None,The text contains two tables and a discussion ...,/Volumes/SJF/CVI_OPACID_SJF_TERMODINAMICKE/Cov...,/Volumes/SJF/CVI_OPACID_SJF_TERMODINAMICKE/str...,None,False


## Exporting Cleaned Data

Finally, let's export the cleaned data back to a JSON file.


In [21]:
# Function to convert DataFrame to JSON-serializable format
def df_to_json_records(df):
    """Convert DataFrame to a list of records suitable for JSON serialization."""
    records = df.to_dict(orient='records')
    return records

# Export the cleaned data
output_file = Path('../output/progress_cleaned_simplified.json')
with open(output_file, 'w', encoding='utf-8') as f:
    json.dump(df_to_json_records(df_clean), f, ensure_ascii=False, indent=4)

df.to_excel('../output/publications.xlsx')

print(f"Cleaned data exported to {output_file}")


Cleaned data exported to ../output/progress_cleaned_simplified.json


## Conclusion

In this notebook, we've demonstrated a simplified approach to:

1. Load and clean metadata from the progress.json file
2. Sanitize titles, authors, and publishers
3. Clean and validate ISBN and DOI values
4. Create a clean DataFrame with single ISBN and DOI columns
5. Check ISBNs against external databases
6. Fetch missing data for books with valid ISBNs
7. Create indicators for ISBNs found in external databases
8. Export the cleaned data for further use

This simplified approach focuses on the most essential tasks while using tqdm for progress bars and evilflowers_importer classes for validation.
